# NYC Apartment Search

## Setup

In [1]:
# Standard library imports
import json
import os
import io
import glob
import warnings
from datetime import datetime

# File and path handling
import pathlib

# HTTP and URL handling
import urllib.parse
import requests

# Data handling and analysis
import pandas as pd
import numpy as np
import geopandas as gpd
from geopandas.tools import sjoin

# Database and SQL handling
import psycopg2
import sqlalchemy as db
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, text
from sqlalchemy.orm import declarative_base, sessionmaker

# Geometry and spatial analysis
import shapely
from shapely.geometry import Point, Polygon
from shapely import wkt
import geoalchemy2 as gdb
from geoalchemy2 import Geometry

# Visualization and plotting
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import contextily as ctx
import seaborn as sns
from PIL import Image

# IPython and widgets
from IPython.display import Image as IPImage, display, HTML
import ipywidgets as widgets
from ipywidgets import interact, IntSlider
from ipywidgets.embed import embed_minimal_html

# Warnings configuration
warnings.filterwarnings('ignore')

In [2]:
# Where data files will be read from/written to - this should already exist
DATA_DIR = pathlib.Path("data")
ZIPCODE_DATA_FILE = DATA_DIR / "zipcodes" / "ZIP_CODE_040114.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

# Download NYC Data
url_311 = 'https://data.cityofnewyork.us/resource/erm2-nwe9.csv'
url_trees = 'https://data.cityofnewyork.us/resource/5rq2-4hqu.csv'
NYC_DATA_APP_TOKEN = "UYsSh8MfAPVog5LPL1G3ySktk"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/"
NYC_DATA_311 = "erm2-nwe9.geojson"
NYC_DATA_TREES = "5rq2-4hqu.geojson"

# create schema.sql file
DB_SCHEMA_FILE = "schema.sql"
# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

In [3]:
# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

## Part 1: Data Preprocessing

### 1. Export Data

In [4]:
def download_nyc_geojson_data(url, app_token, filename, date_field, 
                              start_date, end_date, date_format="%Y-%m-%dT%H:%M:%S", limit=10000):
    """
    Downloads NYC GeoJSON data within a specified date range and writes it to a file.

    This function fetches data from a specified URL using API requests, filtering the data based on a date range. It then writes the data into a file in batches, handling pagination through the 'offset' parameter.

    Parameters:
    - url (str): The URL endpoint for the API from which data is to be fetched.
    - app_token (str): Application token for API access.
    - filename (str): Name of the file where the downloaded data will be saved.
    - date_field (str): The field in the data used to filter by date.
    - start_date (datetime): The start date for the data query.
    - end_date (datetime): The end date for the data query.
    - date_format (str, optional): The format in which dates are represented. Defaults to "%Y-%m-%dT%H:%M:%S".
    - limit (int, optional): The maximum number of records to fetch per request. Defaults to 10000.

    Returns:
    None. The function writes the data to the specified file and prints a message if any request fails.

    The function iterates over batches of data until all records within the specified date range are retrieved and saved to the file. It ensures that the column headers are written only once and handles any HTTP errors encountered during the requests.
    """
    offset = 0
    start_date_str = start_date.strftime(date_format)
    end_date_str = end_date.strftime(date_format)
    date_query = f"$where={date_field} between '{start_date_str}' and '{end_date_str}'"
    
    # set up as the first batch
    first_batch = True  
    while True:
        full_url = f"{url}?$$app_token={app_token}&{date_query}&$limit={limit}&$offset={offset}"
        response = requests.get(full_url)

        if response.status_code == 200:
            data = response.text
            # count the records have been exported
            records_retrieved = data.count('\n') 
            
            # To check if it is the first batch and whether have value
            if first_batch and records_retrieved > 0: 
                # only keep column name in the first batch
                with open(filename, 'w') as file:
                    file.write(data)
                first_batch = False
            elif records_retrieved > 1:  # 
                with open(filename, 'a') as file:
                    # slip the column name
                    file.write(data.split('\n', 1)[1])  
            
            # to check if the data have been exported or not
            if records_retrieved < limit + 1: 
                break
            offset += limit
        else:
            print(f"Failed to download data at offset {offset}: Status code {response.status_code}")
            break

#### 1.1 Download tree 2015 data

In [5]:
# export tree data
download_nyc_geojson_data(
    url=url_trees,
    app_token=NYC_DATA_APP_TOKEN,  
    filename="data/tree_data.csv",
    date_field="created_at",  
    start_date=datetime(2015, 1, 1),
    end_date=datetime(2015, 12, 31),
    date_format="%m/%d/%Y",  
    limit=250000
)

**<span style="color: red;">Test the function 1</span>**

Test for File Existence: This test will check if the tree_data.csv file is created in the specified directory after executing the function.

In [6]:
assert os.path.isfile("data/tree_data.csv"), "File tree_data.csv does not exist"

#### 1.2 Download complaint 311 data by year

The 311 complaint data set is potentially too extensive to import and preprocess directly in the notebook. To manage this, we've opted to process the data on a yearly basis. This involves writing each year's data to a separate CSV file, which we then read individually for preprocessing. Our chosen timeframe extends from January 1, 2015, at 00:00:00 to September 30, 2023, at 23:59:59. This selection aligns with the 2015 tree data, which begins in 2015. Furthermore, we chose the end date to coincide with our query question, which concludes on September 30, 2023.

In [7]:
# Download 311 data from 2015.1.1-2023.9.30 (which is the last date refers in the query)
# Create a new folder to save 311 data by year since the data size is too large
subfolder_name = "311_data"
subfolder_path = os.path.join("data", subfolder_name)
if not os.path.exists(subfolder_path):
    os.makedirs(subfolder_path)

In [8]:
# 311 data for 2015
download_nyc_geojson_data(
    url_311,
    app_token=NYC_DATA_APP_TOKEN,
    filename="data/311_data/311_data_2015.csv",
    date_field="created_date",
    start_date=datetime(2015, 1, 1, 0, 0, 0),
    end_date=datetime(2015, 12, 31, 23, 59, 59),
    date_format="%Y-%m-%dT%H:%M:%S",
    limit=250000
)

In [9]:
# 311 data for 2016
download_nyc_geojson_data(
    url_311,
    app_token=NYC_DATA_APP_TOKEN,
    filename="data/311_data/311_data_2016.csv",
    date_field="created_date",
    start_date=datetime(2016, 1, 1, 0, 0, 0),
    end_date=datetime(2016, 12, 31, 23, 59, 59),
    date_format="%Y-%m-%dT%H:%M:%S",
    limit=250000
)

In [10]:
# 311 data for 2017
download_nyc_geojson_data(
    url_311,
    app_token=NYC_DATA_APP_TOKEN,
    filename="data/311_data/311_data_2017.csv",
    date_field="created_date",
    start_date=datetime(2017, 1, 1, 0, 0, 0),
    end_date=datetime(2017, 12, 31, 23, 59, 59),
    date_format="%Y-%m-%dT%H:%M:%S",
    limit=250000
)

In [11]:
# 311 data for 2018
download_nyc_geojson_data(
    url_311,
    app_token=NYC_DATA_APP_TOKEN,
    filename="data/311_data/311_data_2018.csv",
    date_field="created_date",
    start_date=datetime(2018, 1, 1, 0, 0, 0),
    end_date=datetime(2018, 12, 31, 23, 59, 59),
    date_format="%Y-%m-%dT%H:%M:%S",
    limit=250000
)

In [12]:
# 311 data for 2019
download_nyc_geojson_data(
    url_311,
    app_token=NYC_DATA_APP_TOKEN,
    filename="data/311_data/311_data_2019.csv",
    date_field="created_date",
    start_date=datetime(2019, 1, 1, 0, 0, 0),
    end_date=datetime(2019, 12, 31, 23, 59, 59),
    date_format="%Y-%m-%dT%H:%M:%S",
    limit=250000
)

In [13]:
# 311 data for 2020
download_nyc_geojson_data(
    url_311,
    app_token=NYC_DATA_APP_TOKEN,
    filename="data/311_data/311_data_2020.csv",
    date_field="created_date",
    start_date=datetime(2020, 1, 1, 0, 0, 0),
    end_date=datetime(2020, 12, 31, 23, 59, 59),
    date_format="%Y-%m-%dT%H:%M:%S",
    limit=250000
)

In [14]:
# 311 data for 2021
download_nyc_geojson_data(
    url_311,
    app_token=NYC_DATA_APP_TOKEN,
    filename="data/311_data/311_data_2021.csv",
    date_field="created_date",
    start_date=datetime(2021, 1, 1, 0, 0, 0),
    end_date=datetime(2021, 12, 31, 23, 59, 59),
    date_format="%Y-%m-%dT%H:%M:%S",
    limit=250000
)

In [15]:
# 311 data for 2022
download_nyc_geojson_data(
    url_311,
    app_token=NYC_DATA_APP_TOKEN,
    filename="data/311_data/311_data_2022.csv",
    date_field="created_date",
    start_date=datetime(2022, 1, 1, 0, 0, 0),
    end_date=datetime(2022, 12, 31, 23, 59, 59),
    date_format="%Y-%m-%dT%H:%M:%S",
    limit=250000
)

In [16]:
# 311 data for 2023
download_nyc_geojson_data(
    url_311,
    app_token=NYC_DATA_APP_TOKEN,
    filename="data/311_data/311_data_2023.csv",
    date_field="created_date",
    start_date=datetime(2023, 1, 1, 0, 0, 0),
    end_date=datetime(2023, 9, 30, 23, 59, 59),
    date_format="%Y-%m-%dT%H:%M:%S",
    limit=250000
)

**<span style="color: red;">Test the function 2</span>**

Content: Ensure that the file is not only created but also contains data. This test checks if the file is non-empty.And also for the 311 complaint data the file

In [17]:
directory="data/311_data"
file_count = len([name for name in os.listdir(directory) if os.path.isfile(os.path.join(directory, name))])
assert file_count == 9, f"Expected 9 files, found {file_count}"

### 2. Cleaning & filtering

all the column name is lower case and rename it to the tree(standard), and all zip code need be integer.

|zip        | rent      | complaint   | tree      |description           |
|-----------|-----------|-------------|-----------|----------------------|
|ZIPCODE    |RegionName |incident_zip |zipcode    |five digit postal code|
|           |           |longitude    |longitude  |longitude coordinates |
|           |           |latitude     |latitude   |latitude coordinates  |
|geometry   |           |geometry     |geometry   |geometry  |

**SRID  Normalization: choose 3857 for data visualization**

The SRID (Spatial Reference System Identifier) 3857 is chosen primarily for its compatibility with web mapping and visualization tools, which is excellent for display purposes,. It is the standard coordinate system used by many online mapping services, such as Google Maps and OpenStreetMap. This compatibility facilitates straightforward integration of geospatial data with these platforms, enabling the wide dissemination and easy visualization of geospatial information on the web.

#### 2.1 Clean Zip Code

|zip        |description           |
|-----------|----------------------|
|ZIPCODE    |five digit postal code|
|geometry   |geometry  |

In [18]:
def load_and_clean_zipcodes(zipcode_datafile):
    """
    Loads and cleans a zipcode dataset from a specified file.

    This function performs the following operations on the dataset:
    1. Loads the data from the given file path.
    2. Retains only essential columns, specifically 'ZIPCODE' and 'geometry'.
    3. Removes duplicate entries and invalid data points, ensuring that ZIPCODEs are unique and valid.
    4. Deletes rows with missing ZIPCODE or geometry data.
    5. Converts ZIPCODE from a floating-point to an integer format, retaining only 5-digit ZIPCODEs.
    6. Renames the 'ZIPCODE' column to 'zipcode' and converts all column names to lowercase.
    7. Normalizes the Spatial Reference Identifiers (SRID) of any geometry to the target SRID 'EPSG:3857'.

    Parameters:
    - zipcode_datafile (str): The file path where the zipcode data file is stored.

    Returns:
    - DataFrame: A cleaned and processed pandas DataFrame containing the zipcode data.
    """
    # Load Data
    zip_df = gpd.read_file(zipcode_datafile)

    # 1. Remove unnecessary columns
    keep_columns = ['ZIPCODE', 'geometry']
    clean_zip = zip_df[keep_columns]

    # 2. Remove invalid data points
    # 2.1 Confirm unique_key
    clean_zip = clean_zip.drop_duplicates(subset='ZIPCODE')
    clean_zip = clean_zip[clean_zip['ZIPCODE'].notna() & clean_zip['ZIPCODE'].apply(lambda x: str(x).isdigit())]

    # Define the condition for rows to be removed: remove the data which does not have all location-related data below
    condition = clean_zip['ZIPCODE'].isna() | clean_zip['geometry'].isna()
    # Remove rows based on the condition
    clean_zip = clean_zip[~condition]
    # remove the rows that all cell is nan
    clean_zip = clean_zip.dropna()
    # The zipcode convert it from float to intger
    clean_zip['ZIPCODE'] = clean_zip['ZIPCODE'].astype(int)
    # check if it lasts with 5 digits
    clean_zip = clean_zip[clean_zip['ZIPCODE'].apply(lambda x: str(x).isdigit() and len(str(x)) == 5)]


    # 3. Normalize column names & column types
    # 3.1 Rename the column
    clean_zip.rename(columns={'ZIPCODE': 'zipcode'}, inplace=True)
    # Change all name to lowercase
    clean_zip.columns = [col.lower() for col in clean_zip.columns]

    # 3.2 Reframe the column value type

    # 4. Normalize the Spatial Reference Identifiers (SRID) of any geometry.
    target_srid = "EPSG:3857"
    clean_zip = clean_zip.to_crs(target_srid)

    return clean_zip

**<span style="color: red;">Test the function 3</span>**

To check if the zipcode in zipcode file is already distinct or not, that could help us to extract the zipcode from NYC for the follow dataframe

In [19]:
geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
zip_list_nyc=geodf_zipcode_data['zipcode'].unique()
assert len(zip_list_nyc)==geodf_zipcode_data.shape[0]

#### 2.2 311 Complaint Data Clean

| Field Name     | Description |
|----------------|-------------|
| Unique Key     | Unique identifier of a Service Request (SR) in the open data set. |
| Incident Zip   | Incident location zip code, provided by geo validation. |
| Created Date   | Date SR was created. |
| Complaint Type | This is the first level of a hierarchy identifying the topic of the incident or condition. Complaint Type may have a corresponding Descriptor (below) or may stand alone. |
| longitude      | Geo based Long of the incident location. |
| latitude       | Geo based Lat of the incident location. |

In [20]:
def download_and_clean_311_data(path):
    """
    This function downloads and cleans 311 complaint data.

    The function performs the following steps:
    1. Loads data from the specified CSV file path.
    2. Keeps only essential columns: 'unique_key', 'incident_zip', 'created_date', 'complaint_type', 'longitude', and 'latitude'.
    3. Removes duplicate records and ensures all 'unique_key' values are non-null and numeric.
    4. Excludes rows without complete location-related data (zip code, longitude, latitude).
    5. Ensures latitude and longitude values are within valid ranges.
    6. Removes rows with all NaN values.
    7. Converts 'incident_zip' from float to integer and checks for 5-digit validity.
    8. Filters zip codes to include only those within a predefined list of NYC zip codes.
    9. Renames columns for consistency and converts all column names to lowercase.
    10. Reformats the 'created_date' column to 'YYYY-MM-DD' format.
    11. Transforms the DataFrame into a GeoDataFrame, normalizing spatial reference identifiers (SRID) to EPSG:4326 and then converting to EPSG:3857.

    Parameters:
    path (str): The file path of the 311 data CSV file, e.g., "data/311_data.csv".

    Returns:
    gpd.GeoDataFrame: A cleaned and geospatially normalized DataFrame containing 311 complaint data.
    """
    # Load Data
    complaint_df = pd.read_csv(path)

    # 1. Remove unnecessary columns
    keep_columns = ['unique_key', 'incident_zip', 'created_date', 'complaint_type', 'longitude', 'latitude']
    clean_311 = complaint_df[keep_columns]

    # 2. Remove invalid data points
    # 2.1 Confirm unique_key
    clean_311 = clean_311.drop_duplicates(subset='unique_key')
    clean_311 = clean_311[clean_311['unique_key'].notna() & clean_311['unique_key'].apply(lambda x: str(x).isdigit())]

    # Define the condition for rows to be removed: remove the data which does not have all location-related data below
    condition = clean_311['incident_zip'].isna() | clean_311['longitude'].isna() | clean_311['latitude'].isna()
    # Remove rows based on the condition
    clean_311 = clean_311[~condition]
    # only keep the records with vaild latitude and longitude
    clean_311= clean_311[(clean_311['latitude'].between(-90, 90)) & (clean_311['longitude'].between(-180, 180))]
    # remove the rows that all cell is nan
    clean_311=clean_311.dropna()
    # The zipcode convert it from float to intger
    clean_311['incident_zip'] = clean_311['incident_zip'].astype(int)
    # check if it lasts with 5 digits
    clean_311 = clean_311[clean_311['incident_zip'].apply(lambda x: str(x).isdigit() and len(str(x)) == 5)]
    # only keep the zipcode that in zip_list_nyc
    clean_311 = clean_311[clean_311['incident_zip'].isin(zip_list_nyc)]

    # 3. Normalize column names & column types
    # 3.1 Rename the column
    clean_311.rename(columns={'incident_zip': 'zipcode','created_date':'date','unique_key': 'complaint_id'}, inplace=True)
    # Change all name to lowercase
    clean_311.columns = [col.lower() for col in clean_311.columns]

    # 3.2 Reframe the column value type
    # Create a new column that makes created date only keep year-month-day 2023-11-12
    clean_311['date'] = pd.to_datetime(clean_311['date'])
    clean_311['date'] = clean_311['date'].dt.strftime('%Y-%m-%d')

    # 4. Normalize the Spatial Reference Identifiers (SRID) of any geometry.
    gdf_311 = gpd.GeoDataFrame(clean_311, geometry=gpd.points_from_xy(clean_311.longitude, clean_311.latitude))
    gdf_311.crs = "EPSG:4326"
    # Transform SRID to EPSG:3857 for both GeoDataFrames
    target_srid = "EPSG:3857"
    gdf_311 = gdf_311.to_crs(target_srid)

    return gdf_311


In [21]:
#311 data arguments, as we plan to preprocess the 311 by year
folder_path = "data/311_data"
file_prefix = "311_data_"
years = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']

In [22]:
def clean_concat_311_data(folder_path, file_prefix, years):
    """
    Cleans and merges multiple years of 311 complaint data into a single DataFrame.

    This function iterates over a list of years, loads 311 complaint data from CSV files
    corresponding to each year, cleans the data using the 'download_and_clean_311_data' function,
    and then concatenates all the cleaned data into a single DataFrame.

    Args:
    folder_path (str): The path to the folder containing the CSV files.
    file_prefix (str): The common prefix of the CSV filenames.
    years (list of int): A list of years for which the data is to be processed.

    Returns:
    pd.DataFrame: A pandas DataFrame containing the merged and cleaned data from all specified years.

    The function assumes that the file naming convention is consistent and follows the format of
    'file_prefix' followed by the year and '.csv' (e.g., '311_data_2019.csv' for file_prefix='311_data_' and year=2019).
    It iterates through each year, constructs the filename, and processes the file. After processing all files,
    it concatenates them into a single DataFrame and returns this merged DataFrame.
    """
    cleaned_dfs = []
    for year in years:
        filename = f"{folder_path}/{file_prefix}{year}.csv"
        cleaned_df = download_and_clean_311_data(filename)
        cleaned_dfs.append(cleaned_df)

    # concat the datafiles
    concat_df = pd.concat(cleaned_dfs, ignore_index=True)
    return concat_df